# Energy sensitivity analysis

This notebook performs the sensitivity analysis on UAV energy consumption, evaluating how different energy model assumptions impact route selection and corridor feasibility.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import networkx as nx
from shapely.geometry import LineString

import pandas as pd
import numpy as np
from model_find_paths import connect_distribution_to_postnl

In [ ]:
city = 'breda'
depot = ['Breda']

with open(f'../model/graph_creation/output/{city}.pkl', 'rb') as f:
    G = pickle.load(f)


In [3]:
def run_energy_sensitivity_analysis(G, alphas=[0, 0.5, 1], method="dijkstra"):
    """
    Run energy sensitivity analysis for different alpha values and energy scenarios.
    
    Args:
        G: NetworkX graph
        alphas: List of alpha values to test
        method: Pathfinding method ('dijkstra' or 'astar')
    
    Returns:
        dict: Results organized by alpha value, containing DataFrames with metrics
    """
    
    # Define energy scenarios
    energy_scenarios = [
        ("Base",                      (0.025, 2.03, 0.93)),
        ("Higher horizontal (+20%)",  (0.025 * 1.2, 2.03, 0.93)),
        ("Lower horizontal (-20%)",   (0.025 * 0.8, 2.03, 0.93)),
        ("Higher vertical (+20%)",    (0.025, 2.03 * 1.2, 0.93 * 1.2)),
        ("Lower vertical (-20%)",     (0.025, 2.03 * 0.8, 0.93 * 0.8)),
        ("All higher (+20%)",         (0.025 * 1.2, 2.03 * 1.2, 0.93 * 1.2)),
        ("All lower (-20%)",          (0.025 * 0.8, 2.03 * 0.8, 0.93 * 0.8)),
    ]
    
    all_results = {}
    
    for alpha in alphas:
        print(f"\n{'='*60}")
        print(f"ALPHA = {alpha}")
        print(f"{'='*60}")
        
        alpha_results = {}
        summary_data = []
        
        for scenario_name, (energy_per_m, energy_up, energy_down) in energy_scenarios:
            print(f"\nRunning scenario: {scenario_name}")
            print(f"  - Horizontal energy: {energy_per_m:.4f} Wh/m")
            print(f"  - Vertical up energy: {energy_up:.2f} Wh")
            print(f"  - Vertical down energy: {energy_down:.2f} Wh")
            
            # Run the pathfinding with custom energy parameters
            connected, not_connected, metrics_df = connect_distribution_to_postnl(
                G, 
                alpha=alpha, 
                method=method,
                default_energy_per_m=energy_per_m,
                energy_up_fixed=energy_up,
                energy_down_fixed=energy_down
            )
            
            # Store raw results
            alpha_results[scenario_name] = {
                'connected': connected,
                'not_connected': not_connected,
                'metrics_df': metrics_df
            }
            
            # Calculate summary statistics
            if not metrics_df.empty:
                # Collect edge types
                etypes = []
                for _, _, _, _, etype_array in connected:
                    etypes.extend(etype_array)
                
                summary = {
                    'scenario': scenario_name,
                    'alpha': alpha,
                    'n_paths': len(metrics_df),
                    'n_failed': len(not_connected),
                    # Length statistics
                    'mean_length': metrics_df['length'].mean(),
                    'min_length': metrics_df['length'].min(),
                    'max_length': metrics_df['length'].max(),
                    'std_length': metrics_df['length'].std(),
                    # Risk statistics
                    'mean_risk': metrics_df['risk'].mean(),
                    'min_risk': metrics_df['risk'].min(),
                    'max_risk': metrics_df['risk'].max(),
                    'std_risk': metrics_df['risk'].std(),
                    # Energy statistics
                    'mean_energy': metrics_df['energy'].mean(),
                    'min_energy': metrics_df['energy'].min(),
                    'max_energy': metrics_df['energy'].max(),
                    'std_energy': metrics_df['energy'].std(),
                    # Turns statistics
                    'mean_turns': metrics_df['turns'].mean(),
                    'min_turns': metrics_df['turns'].min(),
                    'max_turns': metrics_df['turns'].max(),
                    'std_turns': metrics_df['turns'].std(),
                    # Height changes statistics
                    'mean_height_changes': metrics_df['height_changes'].mean(),
                    'min_height_changes': metrics_df['height_changes'].min(),
                    'max_height_changes': metrics_df['height_changes'].max(),
                    'std_height_changes': metrics_df['height_changes'].std(),
                    # Unique edge types
                    'n_unique_etypes': len(set(etypes))
                }
            else:
                # No successful paths
                summary = {
                    'scenario': scenario_name,
                    'alpha': alpha,
                    'n_paths': 0,
                    'n_failed': len(not_connected),
                    **{col: np.nan for col in [
                        'mean_length', 'min_length', 'max_length', 'std_length',
                        'mean_risk', 'min_risk', 'max_risk', 'std_risk',
                        'mean_energy', 'min_energy', 'max_energy', 'std_energy',
                        'mean_turns', 'min_turns', 'max_turns', 'std_turns',
                        'mean_height_changes', 'min_height_changes', 'max_height_changes', 'std_height_changes'
                    ]},
                    'n_unique_etypes': 0
                }
            
            summary_data.append(summary)
        
        # Create summary DataFrame for this alpha
        alpha_results['summary_df'] = pd.DataFrame(summary_data)
        all_results[f'alpha_{alpha}'] = alpha_results
        
        # Print summary table for this alpha
        print(f"\nSummary for alpha = {alpha}:")
        print(alpha_results['summary_df'][['scenario', 'n_paths', 'mean_length', 'mean_risk', 'mean_energy']].to_string(index=False))
    
    return all_results


In [4]:
def compare_scenarios_to_base(results_dict, alpha, metric='mean_energy'):
    """
    Compare all scenarios to the base scenario for a given alpha and metric.
    
    Args:
        results_dict: Dictionary returned by run_energy_sensitivity_analysis
        alpha: Alpha value to analyze
        metric: Which metric to compare (e.g., 'mean_energy', 'mean_risk', 'mean_length')
    
    Returns:
        pd.DataFrame: Comparison table
    """
    summary_df = results_dict[f'alpha_{alpha}']['summary_df']
    
    # Get base value
    base_value = summary_df[summary_df['scenario'] == 'Base'][metric].values[0]
    
    # Calculate differences
    comparison = summary_df.copy()
    comparison[f'{metric}_diff'] = comparison[metric] - base_value
    comparison[f'{metric}_pct_change'] = ((comparison[metric] - base_value) / base_value * 100)
    
    return comparison[['scenario', metric, f'{metric}_diff', f'{metric}_pct_change']]

In [5]:
results = run_energy_sensitivity_analysis(G, alphas=[0, 0.5, 1], method="dijkstra")


ALPHA = 0

Running scenario: Base
  - Horizontal energy: 0.0250 Wh/m
  - Vertical up energy: 2.03 Wh
  - Vertical down energy: 0.93 Wh
Distribution points: 1
PostNL points: 64
Alpha: 0, Method: dijkstra
Connected: 157648 → 0 | Length: 2079.4 m
Connected: 157648 → 1 | Length: 5411.2 m
Connected: 157648 → 2 | Length: 7091.7 m
Connected: 157648 → 3 | Length: 4856.1 m
Connected: 157648 → 4 | Length: 3481.7 m
Connected: 157648 → 5 | Length: 3151.6 m
Connected: 157648 → 6 | Length: 4250.9 m
Connected: 157648 → 7 | Length: 2861.1 m
Connected: 157648 → 8 | Length: 2643.8 m
No path: 9
Connected: 157648 → 10 | Length: 3132.6 m
Connected: 157648 → 11 | Length: 5987.9 m
Connected: 157648 → 12 | Length: 3133.2 m
Connected: 157648 → 13 | Length: 5332.2 m
Connected: 157648 → 14 | Length: 6446.6 m
Connected: 157648 → 15 | Length: 6200.1 m
Connected: 157648 → 16 | Length: 4979.6 m
Connected: 157648 → 17 | Length: 3454.6 m
Connected: 157648 → 18 | Length: 3149.3 m
Connected: 157648 → 19 | Length: 2876.

In [24]:
import pandas as pd

def results_to_df(results, include_std=False):
    """
    Convert results dictionary to a single DataFrame with all alphas and scenarios.
    
    Args:
        results: Dictionary from run_energy_sensitivity_analysis
        include_std: Whether to include standard deviation columns (default: False)
    
    Returns:
        pd.DataFrame: Combined DataFrame with all results
    """
    all_data = []
    
    # Iterate through all keys in results
    for key, value in results.items():
        # Extract alpha value from key
        if 'alpha_' in str(key):
            alpha = float(str(key).replace('alpha_', ''))
        else:
            try:
                alpha = float(key)
            except:
                continue
        
        # Get the summary DataFrame for this alpha
        if isinstance(value, dict) and 'summary_df' in value:
            df = value['summary_df'].copy()
            df['alpha'] = alpha
            
            # Drop std columns if not needed
            if not include_std:
                std_cols = [col for col in df.columns if 'std_' in col]
                df = df.drop(columns=std_cols)
            
            all_data.append(df)
    
    # Combine all DataFrames
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        
        # Reorder columns for better readability
        # Put alpha and scenario first, then group metrics
        col_order = ['alpha', 'scenario', 'n_paths', 'n_failed']
        
        # Add metric groups
        metrics = ['length', 'risk', 'energy', 'turns', 'height_changes']
        for metric in metrics:
            for stat in ['mean', 'min', 'max']:
                col = f'{stat}_{metric}'
                if col in combined_df.columns:
                    col_order.append(col)
        
        # Add any remaining columns
        remaining_cols = [col for col in combined_df.columns if col not in col_order]
        col_order.extend(remaining_cols)
        
        # Reorder
        combined_df = combined_df[col_order]
        
        return combined_df
    else:
        return pd.DataFrame()


In [25]:
# Usage
combined_df = results_to_df(results)


In [ ]:
combined_df.to_csv(f'../sensitivity_analysis/output/energy_sensitivity_analysis.csv', index=False)

In [30]:
combined_df

,alpha,scenario,n_paths,n_failed,mean_length,min_length,max_length,mean_risk,min_risk,max_risk,mean_energy,min_energy,max_energy,mean_turns,min_turns,max_turns,mean_height_changes,min_height_changes,max_height_changes,n_unique_etypes
0,0.0,Base,63,1,4172.723638,862.728540,14388.224410,1372.596994,234.491775,6045.833769,113.621424,33.578214,367.655610,123.476190,34,225,5.857143,5.0,7.0,19
1,0.0,Higher horizontal (+20%),63,1,4171.542288,862.728540,14388.224410,1372.067546,234.491775,6045.833769,134.481824,37.891856,439.596732,123.380952,34,225,5.857143,5.0,7.0,19
2,0.0,Lower horizontal (-20%),63,1,4174.022925,862.728540,14388.224410,1376.752061,234.491775,6045.833769,92.751570,29.264571,295.714488,123.984127,34,225,5.825397,5.0,7.0,19
3,0.0,Higher vertical (+20%),63,1,4174.022925,862.728540,14388.224410,1376.752061,234.491775,6045.833769,115.475906,35.980214,369.245610,123.984127,34,225,5.825397,5.0,7.0,19
4,0.0,Lower vertical (-20%),63,1,4171.542288,862.728540,14388.224410,1372.067546,234.491775,6045.833769,111.757002,31.176214,366.065610,123.380952,34,225,5.857143,5.0,7.0,19
5,0.0,All higher (+20%),63,1,4172.723638,862.728540,14388.224410,1372.596994,234.491775,6045.833769,136.345709,40.293856,441.186732,123.476190,34,225,5.857143,5.0,7.0,19
6,0.0,All lower (-20%),63,1,4172.723638,862.728540,14388.224410,1372.596994,234.491775,6045.833769,90.897139,26.862571,294.124488,123.476190,34,225,5.857143,5.0,7.0,19
7,0.5,Base,63,1,5609.272838,912.966797,16084.979907,329.014025,172.084731,597.484605,150.147376,34.834170,410.074498,129.142857,31,276,6.269841,5.0,11.0,19
8,0.5,Higher horizontal (+20%),63,1,5561.364400,912.966797,16076.708822,330.390993,172.084731,604.210676,176.659821,39.399004,490.251265,127.111111,31,272,6.206349,5.0,9.0,19
9,0.5,Lower horizontal (-20%),63,1,5677.137885,912.966797,16104.695345,327.381926,172.084731,597.484605,123.458313,30.269336,330.043907,132.238095,31,278,6.269841,5.0,11.0,19
